### Importing Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import librosa
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.image import resize

In [3]:
import os
os.listdir("/content/drive/MyDrive/Music_Genre_Classification")


['training_hist.json',
 'requirement.txt',
 'music_genre_home.png',
 'Music_Genre_App.py',
 'genres_original',
 'model.h5',
 'Trained_model.keras',
 'Train_Music_Genre_Classifier.ipynb',
 'Test_Music_Genre.ipynb']

In [4]:
model = tf.keras.models.load_model(
    "/content/drive/MyDrive/Music_Genre_Classification/Trained_model.keras"
)


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 26 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
classes = [
    'blues',
    'classical',
    'country',
    'disco',
    'hiphop',
    'pop',
    'rock'
]

### Single Audio Preprocessing

In [6]:
# Load and preprocess audio data
def load_and_preprocess_data(file_path, target_shape=(150, 150)):
    data = []
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    # Perform preprocessing (e.g., convert to Mel spectrogram and resize)
    # Define the duration of each chunk and overlap
    chunk_duration = 4  # seconds
    overlap_duration = 2  # seconds

    # Convert durations to samples
    chunk_samples = chunk_duration * sample_rate
    overlap_samples = overlap_duration * sample_rate

    # Calculate the number of chunks
    num_chunks = int(np.ceil((len(audio_data) - chunk_samples) / (chunk_samples - overlap_samples))) + 1

    # Iterate over each chunk
    for i in range(num_chunks):
                    # Calculate start and end indices of the chunk
        start = i * (chunk_samples - overlap_samples)
        end = start + chunk_samples

                    # Extract the chunk of audio
        chunk = audio_data[start:end]

                    # Compute the Mel spectrogram for the chunk
        mel_spectrogram = librosa.feature.melspectrogram(y=chunk, sr=sample_rate)

                #mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
        mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
        data.append(mel_spectrogram)

    return np.array(data)

### Playing a sound

In [7]:
from IPython.display import Audio
file_path = "/content/drive/MyDrive/Music_Genre_Classification/genres_original/pop/pop.00024.wav"
y, sr = librosa.load(file_path, sr=44100)
Audio(data=y, rate=sr)

Output hidden; open in https://colab.research.google.com to view.

In [8]:
#Processing Test File
X_test = load_and_preprocess_data(file_path)

### Model Prediction

In [9]:
#Model Prediction
def model_prediction(X_test):
    y_pred = model.predict(X_test)
    predicted_categories = np.argmax(y_pred,axis=1)
    unique_elements, counts = np.unique(predicted_categories, return_counts=True)
    #print(unique_elements, counts)
    max_count = np.max(counts)
    max_elements = unique_elements[counts == max_count]
    return max_elements[0]

In [10]:
#Model Prediction
c_index = model_prediction(X_test)
print(f"Model Prediction :: Music Genre --> {classes[c_index]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Model Prediction :: Music Genre --> pop
